## Data management with fiftyone

In [ ]:
! pip install fiftyone

In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt
import fiftyone as fo
import fiftyone.zoo as foz

%matplotlib inline

### Loading datasets form fiftyone zoo

### Exploring samples from a dataset

In [ ]:
# first and last samples

In [ ]:
# get the filepath

In [ ]:
# get the first 10 samples (pandas-like)

### Taking a sliced view of the dataset

In [ ]:
# -> a view


### Reading and visualizing the data in Matplotlib

In [ ]:
fig, axes = plt.subplots(8, 8, figsize=(10, 10))

for i, sample in enumerate(ds.take(64)):
    row, col = i // 8, i % 8
    img = Image.open(sample["filepath"])
    axes[row, col].imshow(img)

[axes[i, j].axis('off') for i in range(8) for j in range(8)]

plt.tight_layout()
plt.show()

### Visualize the data with fiftyone app

In [ ]:
# run the app

# uncomment next line if running as a script:
#session.wait()

In [ ]:
# sorting the labels


### Loading existing datasets

In [ ]:
# list existing datasets

### Deleting a dataset

In [ ]:
# check the list of datasets

## Creating a dataset from local images

```
data/
├── annotations
│   ├── pic01.json
│   ├── pic02.json
│   ├── pic03.json
│   ├── pic04.json
│   ├── ...
│
└── images
    ├── pic01.jpg
    ├── pic02.jpg
    ├── pic03.jpg
    ├── pic04.jpg
    ├── ...
```

In [ ]:
# create a new dataset
ds = fo.Dataset("my-classification-dataset")

ds

In [ ]:
# after creation, use load_dataset()
ds = fo.load_dataset("my-classification-dataset")
ds

#### Creating a new sample

In [ ]:
image_path = "data/images/pic01.jpg"

sample = fo.Sample(filepath=image_path)
sample

#### Adding labels to the sample

In [ ]:
import json

annot_file = "data/annotations/pic01.json"
with open(annot_file, "rt") as f:
    annot = json.load(f)

annot

#### Different types of labels / predictions

 * Classification (multi-class, multi-label)
 * Detection
 * Polylines
 * Keypoints
 

In [ ]:
# check the ground_truth

#### Adding samples to the dataset in a for-loop

In [ ]:
# visualize the dataset
# note: we already have a session


### Bounding-box annotations

In [ ]:
# create a new dataset

images_dir = "data/images"
annots_dir = "data/annotations/"


In [ ]:
# visualize

#### Polyline labels

In [ ]:
images_dir = "data/images"
annots_dir = "data/annotations/"



### Filtering and selecting subsets of the dataset

##### A. Samples with just one object

##### B. Samples with more than 1 objects

##### Cats and dogs 